## Query 1

> Cuál es el estado que más descuentos tiene en total? y en promedio? Supongan que de una direccion del estilo: 3123 Alan Extension Port Andrea, MA 26926, “MA” es el estado.

Los pasos en este caso serían:

1. Importar el dataset de ordenes (data/raw/orders.csv) a un RDD.
2. Filtrar aquellas ordenes cuyos datos de dirección y/o descuentos no sean válidos.
3. Parsear los números a float para poder operar con ellos.
4. Mapear los datos de dirección (shipping y billing) para extraer el estado.
5. Cachear el RDD resultante para evitar recalcularlo en cada subconsulta.
6. Ahora para cada dirección (shipping, billing) aplicamos un agrupado haciendo:
    1. Un filter de los estados vacíos
    2. Un mapeo de las ordenes para que la key sea el estado de la dirección
    3. Un reduceByKey para calcular los totales
    4. Un cache para reutilizar los cálculos
    5. Finalmente para cada métrica hacemos:
        1. Un map para quedarnos/cacular la métrica que nos interesa
        2. Un reduce para quedarnos con aquella que sea mayor

In [5]:
import re
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("DescuentosEstado").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [7]:
df = spark.read.csv("../../data/raw/orders.csv", header=True, inferSchema=True)
rdd = df.rdd

cleaned = (
    rdd
    .filter(lambda r: r.shipping_address and r.billing_address and r.total_amount is not None)
    .map(lambda r: {
        "shipping_address": r.shipping_address,
        "billing_address": r.billing_address,
        "total_amount": float(r.total_amount),
        "discount_amount": float(getattr(r, "discount_amount", 0.0) or 0.0),
    })
)

STATE_REGEX = r",\s*([A-Z]{2})\s+\d{5}"

def extract_state(address):
    if not address:
        return None
    match = re.search(STATE_REGEX, address)
    return match.group(1) if match else None

processed = cleaned.map(
    lambda x: {
        "state_shipping": extract_state(x["shipping_address"]),
        "state_billing": extract_state(x["billing_address"]),
        "total_amount": x["total_amount"],
        "discount_absolute": x["total_amount"] * x.get("discount_amount", 0.0) / 100.0,
    }
).cache()


In [9]:
def find_max_discounts(data, state_key):
    grouped = (
        data.filter(lambda x: x[state_key] is not None)
        .map(
            lambda x: (
                x[state_key],
                {
                    "count": 1,
                    "sum_abs": x["discount_absolute"],
                },
            )
        )
        .reduceByKey(
            lambda a, b: {
                "count": a["count"] + b["count"],
                "sum_abs": a["sum_abs"] + b["sum_abs"],
            }
        )
        .map(
            lambda x: {
                "state": x[0],
                "count": x[1]["count"],
                "total_discount": x[1]["sum_abs"],
            }
        )
    ).cache()

    # Reduce to find max total discount
    max_total = grouped.map(lambda x: (x["state"], x["total_discount"])).reduce(
        lambda a, b: a if a[1] > b[1] else b
    )

    # Reduce to find max average discount
    max_avg = grouped.map(
        lambda x: (
            x["state"],
            x["total_discount"] / x["count"] if x["count"] > 0 else 0,
        )
    ).reduce(lambda a, b: a if a[1] > b[1] else b)

    return max_total, max_avg


max_total_shipping, max_avg_shipping = find_max_discounts(processed, "state_shipping")
max_total_billing, max_avg_billing = find_max_discounts(processed, "state_billing")

print("Shipping - Estado con máximo descuento total:", max_total_shipping)
print("Shipping - Estado con máximo descuento promedio:", max_avg_shipping)
print("Billing - Estado con máximo descuento total:", max_total_billing)
print("Billing - Estado con máximo descuento promedio:", max_avg_billing)


Shipping - Estado con máximo descuento total: ('UT', 134728.09174800006)
Shipping - Estado con máximo descuento promedio: ('UT', 2.754049299836469)
Billing - Estado con máximo descuento total: ('MO', 134500.404522)
Billing - Estado con máximo descuento promedio: ('MO', 2.7259911739359546)
